In [171]:
import pandas as pd
import numpy as np

In [2]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [9]:
iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [10]:
set(iris['species'])

{'setosa', 'versicolor', 'virginica'}

## 1. Subset Data

In [23]:
output = iris[iris['sepal_length'] > 5.0]
output = iris[(iris['sepal_length'] > 5.0) & (iris['species'] == 'versicolor')]
output = iris[(iris['sepal_length'] > 5.0) & (iris['species'] != 'versicolor')]
output = iris[(iris['sepal_length'] > 5.0) & (~iris['species'].isin(['versicolor', 'setosa']))]

In [95]:
output = iris.sample(frac=0.5) # randomly sample 50% of rows
output = iris.sample(n=50) # randomly sample 50 rows
output = iris.nlargest(10, 'sepal_length') # 10 largest in sepal length
output = iris.nsmallest(10, 'sepal_length') # 10 smallest by sepal length

### loc vs iloc

In [47]:
# loc vs iloc
# label-location-based vs integer-location-based

# iris.loc['row_label', 'column_label']
# iris.iloc[row_position, column_position]

# --- loc ---- #
# Single column
iris.loc[:, 'sepal_length']

# Multiple columns
iris.loc[:, ['sepal_length', 'species']]

# Single row; you don't necessarily need to wrap with a list, but this will perserve
# df format
iris.loc[1, :]
iris.loc[[1], :]

# Multiple rows
iris.loc[1:3, :]

# Rows + Column selection
iris.loc[1:3, ['sepal_length', 'species']]

# ---- iloc ------ #
# Single column
iris.iloc[:, 3]

# Multiple columns
iris.iloc[:, 3:5]

# Single row
iris.iloc[1, :]

# Multiple rows
iris.iloc[1:5, :]

# Multiple rows and columns
iris.iloc[1:5, 3:5]

,petal_width,species
1,0.2,setosa
2,0.2,setosa
3,0.2,setosa
4,0.2,setosa


In [92]:
# Data.table like filtering with loc
iris.loc[iris['petal_width'] > 0.7, ['petal_length','species']]

# Replacing data aka dt[x1 > 10, ':=' (x3 = 'replace_string')]
output = iris.copy()
output.loc[((output['petal_width'] > 0.5) & (output['species'] == 'setosa')), 'species'] = 'walrus'
output.loc[(output['petal_width'] > 0.5) & (output['species'] == 'walrus'), ['petal_width','species']]

,petal_width,species
43,0.6,walrus


## 2. Group by --  (aggregations + Summary functions) & (window functions)

In [110]:
# Summary functions in pandas
# Sum
iris['sepal_length'].sum()

# Count excluding Na and NULL (alternative to length)
iris['sepal_length'].count()

# Median
iris['sepal_length'].median()

# Quantile, get the specific break points
print(iris['sepal_length'].quantile([0.25, 0.75]))

# Min, Max
iris['sepal_length'].min()
iris['sepal_length'].max()

# Mean
iris['sepal_length'].mean()

# Variance
iris['sepal_length'].var()

# Stddev
iris['sepal_length'].std()


0.25    5.1
0.75    6.4
Name: sepal_length, dtype: float64


0.8280661279778629

In [113]:
# Syntax is straightforward
# df.groupby(by = 'col').fun()

# Summation group by
# Chain the groupby syntax and the summary functions above
iris.groupby(by='species').sum()
iris.groupby(by='species').std()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,0.352490,0.379064,0.173664,0.105386
versicolor,0.516171,0.313798,0.469911,0.197753
virginica,0.635880,0.322497,0.551895,0.274650


In [195]:
# Different aggregations
test = iris.groupby(by='species').agg({"sepal_length": [np.mean, np.sum]}).reset_index()
test.columns = ['species','sepal_mean', 'sepal_sum']
test

,species,sepal_mean,sepal_sum
0,setosa,5.006,250.3
1,versicolor,5.936,296.8
2,virginica,6.588,329.4


In [197]:
(iris.groupby(by='species')
     .agg({"sepal_length": [np.mean, np.sum]})
     .reset_index())

species sepal_length       
                      mean    sum
0      setosa        5.006  250.3
1  versicolor        5.936  296.8
2   virginica        6.588  329.4

In [145]:
# Group by offsetting + cumulative functions
# do sort_values(['group_name', 'value_col'], ascending=[True, True])
output = iris.copy()
output['shifted_petal_width'] = iris.groupby('species')['petal_width'].shift(1)
output[output['species']=='virginica'].head()

# Cumulative examples
output['cumulative_petal_width'] = iris.groupby('species')['petal_width'].cumsum()
output['cumprod_petal_width'] = iris.groupby('species')['petal_width'].cumprod()

# Ranking
# allows for ties, no skipping 1, 2, 2, 3
output['petal_width_rank'] = iris.groupby('species')['petal_width'].rank(method='dense') 
# allows for ties, yes skipping 1, 2, 2, 4
output['petal_width_rank'] = iris.groupby('species')['petal_width'].rank(method='min')  
# no ties, first value it sees will get that number, 1, 2, 3, 4
output['petal_width_rank'] = iris.groupby('species')['petal_width'].rank(method='first')  


In [146]:
output[output['species']=='virginica'].sort_values('petal_width_rank').head()


,sepal_length,sepal_width,petal_length,petal_width,species,shifted_petal_width,cumulative_petal_width,cumprod_petal_width,petal_width_rank
134,6.1,2.6,5.6,1.4,virginica,1.5,69.2,1.644787e+10,1.0
119,6.0,2.2,5.0,1.5,virginica,2.3,40.9,1.374825e+06,2.0
133,6.3,2.8,5.1,1.5,virginica,2.2,67.8,1.174848e+10,3.0
129,7.2,3.0,5.8,1.6,virginica,2.1,60.2,9.368800e+08,4.0
106,4.9,2.5,4.5,1.7,virginica,2.1,14.3,1.410186e+02,5.0


In [163]:
#---- Window Functions ---- #
# df.groupby()[column].window().func

# expanding windows
# Set a minimum window e.g. 3 periods, then it will grow after
# NA, NA, NA, 3 obs, 4 obs, 5 obs etc....
# unlike cumulative we get to set a minimum
output = iris.copy()
# New version on pandas requires us to reset index
output['expanding_petal_width_mean'] = output.groupby(['species'])['petal_width'].expanding(3).mean().reset_index(drop=True)

# rolling windows
output['rolling_petal_width_mean'] = output.groupby(['species'])['petal_width'].rolling(3).mean().reset_index(drop=True)


## 3. Reshaping Data

In [180]:
# Pivot
# From rows to columns
# Syntax: pd.pivot_table(df, values=number_column, index=[list_of_columns_to_preserve],
#   columns=[list_of_columns to pivot], aggfunc=np.sum)
output = iris.copy()
output['group_two'] = 'A'
output.loc[output['species'] == 'virginica', 'group_two'] = 'B'

pd.pivot_table(output, values='petal_width', columns=['species'], aggfunc=np.sum)
output2 = pd.pivot_table(output, values='petal_width', index=['group_two'], columns=['species'], aggfunc=np.sum)
output2 = output2.reset_index()
output2

species,group_two,setosa,versicolor,virginica
0,A,12.3,66.3,NaN
1,B,NaN,NaN,101.3


In [181]:
# Melt
# From columns to rows
# Syntax: pd.melt(df, id_vars=['group_two'],      #what you want to keep as is
#                     value_vars = ['setosa', 'veriscolor', 'virginica']      # columns you want to unpivot
#                     var_name='species'      # name of the new column
#                     value_name = 'total_petal_width'   # name of the unpivoted number values 
#)

pd.melt(output2, id_vars=['group_two'], value_vars=['setosa', 'versicolor', 'virginica'],
 var_name='species', value_name='total_petal_width')

,group_two,species,total_petal_width
0,A,setosa,12.3
1,B,setosa,NaN
2,A,versicolor,66.3
3,B,versicolor,NaN
4,A,virginica,NaN
5,B,virginica,101.3


In [ ]:
# Concat
# Append row (aka union all or rbind())


## Date Logic

In [ ]:
from datetime import datetime
(df['created_at'] - df['prev_value']).dt.days
# dt.months
# dt.years
# dt.to_period('M')